In [3]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datestart='2025-01-02'
dateend='2024--01'
datefilter=''

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
df=pd.read_sql_query('Select Distinct * from allmatches',con=devengine)
# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]

# Populate Set Winner columns based on set scores and Completed_Sets
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    w_col = f"w{set_num}"
    l_col = f"l{set_num}"
    
    # Ensure the column exists
    if w_col in df.columns and l_col in df.columns:
        # Determine the winner for the set
        df[set_winner_col] = df.apply(
            lambda row: row["Player_1"] if row[w_col] > row[l_col] else row["Player_2"], axis=1
        )
    else:
        df[set_winner_col] = None  # Initialize with None if scores don't exist

    # Clear values for sets beyond the number of completed sets
    df[set_winner_col] = df.apply(
        lambda row: row[set_winner_col] if set_num <= row["Completed_Sets"] else None, axis=1
    )
df["fav_odds"] = df.apply(
    lambda row: min(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
df["dog_odds"] = df.apply(
    lambda row: max(row["Player_1_Odds"], row["Player_2_Odds"]), axis=1
)
# Create columns for fav_player and dog_player based on odds
df["fav_player"] = df.apply(
    lambda row: row["Player_1"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_2"], axis=1
)
df["dog_player"] = df.apply(
    lambda row: row["Player_2"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_1"], axis=1
)

# Adjust the odds columns to reflect the favorite and underdog
df["fav_odds"] = df.apply(
    lambda row: row["Player_1_Odds"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_2_Odds"], axis=1
)
df["dog_odds"] = df.apply(
    lambda row: row["Player_2_Odds"] if row["Player_1_Odds"] == row["fav_odds"] else row["Player_1_Odds"], axis=1
)

# Adjust the Set Winner columns to reflect the favorite and underdog
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    
    # Reorganize the Set Winner columns based on the favorite and underdog
    df[set_winner_col] = df.apply(
        lambda row: row["fav_player"] if row[set_winner_col] == row["Player_1"] else row["dog_player"], axis=1
    )

# Ensure Winner_Sets and Loser_Sets are integers
df["Winner_Sets"] = df["Winner_Sets"].astype(int)
df["Loser_Sets"] = df["Loser_Sets"].astype(int)

# Calculate the total number of completed sets
df["Completed_Sets"] = df["Winner_Sets"] + df["Loser_Sets"]

# Populate Set Winner columns based on set scores and Completed_Sets
for set_num in range(1, 6):
    set_winner_col = f"Set {set_num} Winner"
    w_col = f"w{set_num}"
    l_col = f"l{set_num}"
    
    # Ensure the column exists
    if w_col in df.columns and l_col in df.columns:
        # Determine the winner for the set
        df[set_winner_col] = df.apply(
            lambda row: "Fav" if row[w_col] > row[l_col] else "Dog", axis=1
        )
    else:
        df[set_winner_col] = None  # Initialize with None if scores don't exist

    # Clear values for sets beyond the number of completed sets
    df[set_winner_col] = df.apply(
        lambda row: row[set_winner_col] if set_num <= row["Completed_Sets"] else None, axis=1
    )

df["Match_Type"] = df.apply(
    lambda row: "Best of 5" if row["Winner_Sets"] + row["Loser_Sets"] > 3 else "Best of 3", axis=1
)
df

,level_0,index,Date,Sex,Tournament,Player_1,Player_2,Player_1_Odds,Player_2_Odds,Surface,...,Set 1 Winner,Set 2 Winner,Set 3 Winner,Set 4 Winner,Set 5 Winner,fav_odds,dog_odds,fav_player,dog_player,Match_Type
0,0.0,0.0,2023-01-02,Mens,Oeiras,Alibek Kachmazov,Timo Legout,1.41,2.72,Hard,...,Fav,Fav,None,None,None,1.41,2.72,Alibek Kachmazov,Timo Legout,Best of 3
1,1.0,1.0,2023-01-02,Mens,Oeiras,Rudolf Molleker,Simon Beaupain,1.30,3.28,Hard,...,Fav,Fav,None,None,None,1.30,3.28,Rudolf Molleker,Simon Beaupain,Best of 3
2,2.0,2.0,2023-01-02,Mens,Oeiras,Joris De Loore,Oscar Jose Gutierrez,1.22,3.89,Hard,...,Fav,Fav,None,None,None,1.22,3.89,Joris De Loore,Oscar Jose Gutierrez,Best of 3
3,3.0,3.0,2023-01-02,Mens,Oeiras,Sebastian Fanselow,Fabio Coelho,1.16,4.48,Hard,...,Fav,Fav,None,None,None,1.16,4.48,Sebastian Fanselow,Fabio Coelho,Best of 3
4,4.0,4.0,2023-01-02,Mens,Oeiras,Edan Leshem,Henrique Rocha,1.35,2.94,Hard,...,Dog,Fav,Fav,None,None,1.35,2.94,Edan Leshem,Henrique Rocha,Best of 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36058,NaN,NaN,2025-01-02,Womens,Brisbane3,Kimberly Birrell,Anastasia Potapova,2.51,1.52,Hard,...,Fav,Fav,None,None,None,1.52,2.51,Anastasia Potapova,Kimberly Birrell,Best of 3
36059,NaN,NaN,2025-01-02,Womens,Auckland20,Hailey Baptiste,Anna Burrage Jodie,1.54,2.47,Hard,...,Dog,Fav,Fav,None,None,1.54,2.47,Hailey Baptiste,Anna Burrage Jodie,Best of 3
36060,NaN,NaN,2025-01-02,Womens,Auckland20,Alycia Parks,Greet Minnen,1.92,1.87,Hard,...,Fav,Fav,None,None,None,1.87,1.92,Greet Minnen,Alycia Parks,Best of 3
36061,NaN,NaN,2025-01-02,Womens,Auckland20,Robin Montgomery,Nao Hibino,1.70,2.14,Hard,...,Fav,Fav,None,None,None,1.70,2.14,Robin Montgomery,Nao Hibino,Best of 3


In [2]:
# Filter the DataFrame based on the given conditions
filtered_df = df[(df["fav_odds"] < 1.8) & (df["Match_Type"] == "Best of 3")& (df["Sex"] != "Mens") & (((df["w1"] == "7")&(df["l1"] == "6"))|((df["l1"] == "7")&(df["w1"] == "6")))]

# Count the number of times Set 2 Winner also wins Set 3
set_2_winner_wins_set_3_count = filtered_df[filtered_df["Set 1 Winner"] == filtered_df["Set 2 Winner"]].shape[0]

# Count the total number of filtered matches
total_filtered_matches = filtered_df.shape[0]

# Calculate the percentage
set_2_winner_wins_set_3_percentage = (set_2_winner_wins_set_3_count / total_filtered_matches) * 100

# Display the results
print(f"Total Filtered Matches: {total_filtered_matches}")
print(f"Matches where Set 2 Winner wins Set 3: {set_2_winner_wins_set_3_count}")
print(f"Percentage: {set_2_winner_wins_set_3_percentage:.2f}%")
filtered_df[['w1']]


Total Filtered Matches: 983
Matches where Set 2 Winner wins Set 3: 548
Percentage: 55.75%


,w1
325,7
332,6
358,7
366,7
426,7
...,...
36024,6
36029,7
36051,6
36052,7
